In [10]:
using Distributions
using DataFrames
using CSV
using Random

In [11]:
# Seed
rng = MersenneTwister(2137)

MersenneTwister(2137)

In [12]:
function calculate_parameters(number_of_persons::Int64, mean_arrival_rate::Float64, mean_service_rate::Float64)
    return calculated_parameters = Dict(
        'n' => number_of_persons,
        "mean_arrival_rate" => mean_arrival_rate,
        "mean_service_rate" => mean_service_rate,
        "mean_interarrival_time" => 1.0 / mean_arrival_rate,
        "mean_service_time" => 1.0 / mean_service_rate
    )
end

calculate_parameters (generic function with 1 method)

In [13]:
function time_dataframe(calculated_parameters::Dict, interarrival_times, arrival_times, service_times)
    # Unpacking parameters
    n = calculated_parameters['n']
    
    # Creating DataFrame
    time_df = DataFrame(
        interarrival_time = interarrival_times,
        arrival_time = arrival_times,
        service_time = service_times,
        service_start_time = zeros(n),
        service_end_time = zeros(n),
        wait_time = zeros(n)
        )
    
    # Calculating and updating DataFrame parameters
    time_df[1, :service_start_time] = time_df[1, :arrival_time]
    time_df[1, :service_end_time] = time_df[1, :service_start_time] + time_df[1, :service_time] 
    
    for i in 2:n
        time_df[i, :service_start_time] = max(time_df[i, :arrival_time], time_df[i-1, :service_end_time])
        time_df[i, :service_end_time] = time_df[i, :service_start_time] + time_df[i, :service_time]
    end    
    
    time_df[:, :wait_time] = time_df[:, :service_start_time] - time_df[:, :arrival_time]
    
    CSV.write("time_df.csv", time_df)
    
    return time_df
end

time_dataframe (generic function with 1 method)

In [14]:
function event_dataframe(calculated_parameters::Dict, time_df::DataFrame)
    # Unpacking parameters
    n = calculated_parameters['n']
    arrival_times = time_df[:, :arrival_time]
    service_start_times = time_df[:, :service_start_time]
    service_end_times = time_df[:, :service_end_time]
    
    # Creating DataFrame
    event_df = DataFrame(
        persons_in_model = 0.0,
        persons_in_queue = 0.0,
        event_time = 0.0,
        )
    
    # Parameters for the loop
    persons_in_model = 0
    persons_in_queue = 0
    event_time = 0
    end_index = 1
    arrival_index = 1
    start_index = 1
    
    # Loop
    while end_index ≤ n
        
        # Bypassing "Trying to access n+1 element of n element vector"
        # For large simulations replace 10^10 with external library Infinity.jl
        if arrival_index ≤ n
            arrival_event_time = arrival_times[arrival_index]
        else 
            arrival_event_time = 10^10
        end 
        
        if start_index ≤ n
            start_event_time = service_start_times[start_index]
        else
            start_event_time = 10^10
        end 
        
        end_event_time = service_end_times[end_index]
        
        
        # Actual DataFrame Row Calculation
        if (arrival_event_time ≤ start_event_time) && (arrival_event_time ≤ end_event_time)
            event_time = arrival_event_time
            queue_change = 1
            model_change = 1
            arrival_index += 1
        elseif (start_event_time ≤ arrival_event_time) && (start_event_time ≤ end_event_time)
            event_time = start_event_time
            queue_change = -1
            model_change = 0
            start_index += 1
        else 
            event_time = end_event_time
            queue_change = 0
            model_change = -1
            end_index += 1
        end
        
        # Update parameters and save
        persons_in_model = persons_in_model + model_change
        persons_in_queue = persons_in_queue + queue_change  
        
        push!(
            event_df,
            [persons_in_model,
            persons_in_queue,
            event_time] 
            )
    end
    CSV.write("event_df.csv", event_df)
    return event_df
end

event_dataframe (generic function with 1 method)

In [15]:
function start_simulation(number_of_persons::Int64, mean_arrival_rate::Float64, mean_service_rate::Float64)
    # Calculating parameters
    calculated_parameters = calculate_parameters(number_of_persons, mean_arrival_rate, mean_service_rate)
    
    # Unpacking parameters
    n = calculated_parameters['n']
    mean_interarrival_time = calculated_parameters["mean_interarrival_time"]
    mean_service_time = calculated_parameters["mean_service_time"]
    
    # Calculating distributions
    interarrival_times = rand(rng, Exponential(mean_interarrival_time), n)
    arrival_times = cumsum(interarrival_times)
    service_times = rand(rng, Exponential(mean_service_time), n)
    
    # Calculating DataFrames
    time_df = time_dataframe(calculated_parameters, interarrival_times, arrival_times, service_times)
    event_df = event_dataframe(calculated_parameters, time_df)
    return time_df
end

start_simulation (generic function with 1 method)

In [16]:
time_df = start_simulation(100, 60.0, 40.0)

,interarrival_time,arrival_time,service_time,service_start_time,service_end_time
,Float64,Float64,Float64,Float64,Float64
1,0.00395125,0.00395125,0.00606062,0.00395125,0.0100119
2,0.00157941,0.00553066,0.0018451,0.0100119,0.011857
3,0.00143926,0.00696993,0.0305746,0.011857,0.0424315
4,0.0087202,0.0156901,0.0578195,0.0424315,0.100251
5,0.0170744,0.0327645,0.0502043,0.100251,0.150455
6,0.000164845,0.0329294,0.028406,0.150455,0.178861
7,0.00862829,0.0415577,0.0490402,0.178861,0.227902
8,0.00406946,0.0456271,0.0964417,0.227902,0.324343
9,0.00781482,0.0534419,0.0185782,0.324343,0.342921
